In [1]:
import csv
from lib.import_lang_data import *
from lib.lang_featurizers import *
# import glob
# import numpy as np
import pandas as pd
import re
from collections import Counter
from sklearn.base import TransformerMixin
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from textblob import TextBlob

In [2]:
# read data into lang_data, lang_results
languages = ['gcc', 'c', 'csharp', 'sbcl', 'clojure', 'ghc' 'java', 'javascript',
             'ocaml', 'perl', 'php', 'hack', 'py', 'python3', 'jruby', 'yarv', 'rb',
             'scala', 'racket']
lang_data, lang_results = read_polyglot(languages)
lang_info = pd.DataFrame(lang_results)
lang_info[0].value_counts()

gcc           58
scala         43
csharp        41
yarv          39
clojure       38
python3       36
ocaml         35
perl          34
sbcl          34
jruby         34
php           29
racket        29
hack          26
javascript    25
c              1
dtype: int64

In [3]:
lang_info = match_extensions(lang_info)
lang_results = list(lang_info[0])
lang_info[0].value_counts()

C              59
PHP            55
Scala          43
C#             41
Ruby           39
Clojure        38
Python         36
OCaml          35
jRuby          34
Common Lisp    34
Perl           34
Scheme         29
JavaScript     25
dtype: int64

In [9]:
lang_featurizer = make_union(
    BagOfWordsFeaturizer(20),
    FunctionFeaturizer(num_nil,
                       percentage_of_punctuation)
)

X_train, X_test, y_train, y_test = train_test_split(lang_data, lang_results)

pipe = make_pipeline(lang_featurizer, DecisionTreeClassifier())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.65873015873015872

In [10]:
print(classification_report(pipe.predict(X_test), y_test))

             precision    recall  f1-score   support

          C       0.63      0.71      0.67        17
         C#       0.73      0.57      0.64        14
    Clojure       0.83      0.56      0.67         9
Common Lisp       0.57      0.80      0.67         5
 JavaScript       0.67      0.57      0.62         7
      OCaml       1.00      0.67      0.80         9
        PHP       0.83      0.67      0.74        15
       Perl       0.38      0.71      0.50         7
     Python       1.00      1.00      1.00         7
       Ruby       0.38      0.56      0.45         9
      Scala       1.00      1.00      1.00         8
     Scheme       0.67      0.80      0.73        10
      jRuby       0.17      0.11      0.13         9

avg / total       0.69      0.66      0.66       126



## Now to test with the assignment tests

In [11]:
def read_tests():
    X = []
    y = None
    files = glob.glob('test/*')
    for file in files:
        with open(file) as f:
            X.append(f.read())

    with open('test.csv') as f:
        y = f.read()
    
    inter = y.split('\n')
    
    y_out = []
    for pair in inter:
        y_out.append(pair.split(','))
    
    y_out = list(pd.DataFrame(y_out).pop(1))[:-1]
    
    y_df = match_extensions(pd.DataFrame(y_out))
    y_out = list(y_df[0])
    
    return X, y_out

In [12]:
X_assignment_test, y_assignment_test = read_tests()

In [13]:
pipe.score(X_assignment_test, y_assignment_test)

0.0625

In [14]:
print(classification_report(pipe.predict(X_assignment_test), y_assignment_test))

             precision    recall  f1-score   support

          C       0.00      0.00      0.00        11
    Clojure       0.00      0.00      0.00         1
Common Lisp       0.00      0.00      0.00         4
    Haskell       0.00      0.00      0.00         0
       Java       0.00      0.00      0.00         0
 JavaScript       0.00      0.00      0.00         0
      OCaml       0.00      0.00      0.00         1
        PHP       0.00      0.00      0.00         0
       Perl       0.00      0.00      0.00         5
     Python       0.50      0.25      0.33         8
       Ruby       0.00      0.00      0.00         2
      Scala       0.00      0.00      0.00         0
     Scheme       0.00      0.00      0.00         0
        TCL       0.00      0.00      0.00         0

avg / total       0.12      0.06      0.08        32



/Users/ahartz1/TIY/programming-language-classifier/.direnv/python-3.4.3/lib/python3.4/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/ahartz1/TIY/programming-language-classifier/.direnv/python-3.4.3/lib/python3.4/site-packages/sklearn/metrics/classification.py:960: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
